# 🧪 Model Experiments Notebook
## Build and test your CNN architecture

In this notebook, you'll:
1. Load your data using PyTorch
2. Build your CNN model
3. Test the forward pass
4. Experiment with different architectures

## 1. Setup & Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pandas as pd
import os

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Configuration

In [ ]:
# Hyperparameters - experiment with these!
IMAGE_SIZE = 32
NUM_CLASSES = 28  # 28 Arabic letters
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 10  # Start small for experiments

# Data paths - UPDATE THESE!
DATA_DIR = "../data/raw"
CSV_FILE = "labels.csv"  # Adjust to your file name

## 3. Build the Dataset Class

This is **PILLAR 1** - implement your data loading here!

In [ ]:
class ArabicLetterDataset(Dataset):
    """
    TODO: Complete this class!

    You need to:
    1. Load the CSV file in __init__
    2. Return the number of samples in __len__
    3. Load and return one (image, label) pair in __getitem__
    """

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file: Path to CSV with image paths and labels
            root_dir: Directory containing images
            transform: Transforms to apply to images
        """
        # TODO: Load CSV file
        # self.data_frame = pd.read_csv(csv_file)
        # self.root_dir = root_dir
        # self.transform = transform
        pass

    def __len__(self):
        # TODO: Return number of samples
        # return len(self.data_frame)
        pass

    def __getitem__(self, idx):
        # TODO: Load image and label
        # 1. Get image path from CSV
        # 2. Load image with PIL
        # 3. Apply transforms
        # 4. Get label from CSV
        # 5. Return (image, label)
        pass

In [ ]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Scale to [-1, 1]
])

print("Transforms defined:")
print(transform)

In [ ]:
# TODO: Uncomment when your Dataset class is complete

# # Create dataset
# full_dataset = ArabicLetterDataset(
#     csv_file=os.path.join(DATA_DIR, CSV_FILE),
#     root_dir=DATA_DIR,
#     transform=transform
# )
#
# # Split into train and validation
# train_size = int(0.8 * len(full_dataset))
# val_size = len(full_dataset) - train_size
# train_dataset, val_dataset = torch.utils.data.random_split(
#     full_dataset, [train_size, val_size]
# )
#
# # Create DataLoaders
# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
#
# print(f"Training samples: {len(train_dataset)}")
# print(f"Validation samples: {len(val_dataset)}")

## 4. Build the CNN Model

This is **PILLAR 2** - design your neural network!

In [ ]:
class ArabicCNN(nn.Module):
    """
    Convolutional Neural Network for Arabic Letter Recognition.

    Architecture:
    - Conv Block 1: Conv2d -> ReLU -> MaxPool
    - Conv Block 2: Conv2d -> ReLU -> MaxPool
    - Fully Connected: Flatten -> Linear -> ReLU -> Linear
    """

    def __init__(self):
        super(ArabicCNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        # Pooling layer
        self.pool = nn.MaxPool2d(2, 2)

        # Fully connected layers
        # After 2 pools: 32 -> 16 -> 8, with 64 channels
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, NUM_CLASSES)

    def forward(self, x):
        # Conv block 1
        x = self.pool(F.relu(self.conv1(x)))

        # Conv block 2
        x = self.pool(F.relu(self.conv2(x)))

        # Flatten
        x = x.view(x.size(0), -1)

        # Fully connected
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [ ]:
# Create and inspect the model
model = ArabicCNN()
print("Model Architecture:")
print(model)

# Count parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal trainable parameters: {total_params:,}")

In [ ]:
# Test the forward pass with dummy data
dummy_input = torch.randn(1, 1, 32, 32)  # (batch=1, channels=1, height=32, width=32)
dummy_output = model(dummy_input)

print(f"Input shape: {dummy_input.shape}")
print(f"Output shape: {dummy_output.shape}")
print(f"Expected: (1, {NUM_CLASSES})")
print(f"\n✓ Forward pass successful!" if dummy_output.shape == (1, NUM_CLASSES) else "✗ Shape mismatch!")

## 5. Test Training Loop (Mini Experiment)

Let's test that everything works with a small training loop.

In [ ]:
# Setup for training
model = ArabicCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(f"Model moved to: {device}")
print(f"Loss function: CrossEntropyLoss")
print(f"Optimizer: Adam (lr={LEARNING_RATE})")

In [ ]:
# Test with fake data to verify the training loop works
print("Testing training loop with fake data...\n")

# Create fake data
fake_images = torch.randn(BATCH_SIZE, 1, IMAGE_SIZE, IMAGE_SIZE).to(device)
fake_labels = torch.randint(0, NUM_CLASSES, (BATCH_SIZE,)).to(device)

# Training step
model.train()
for epoch in range(3):  # Just 3 fake epochs
    # Forward pass
    outputs = model(fake_images)
    loss = criterion(outputs, fake_labels)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Calculate accuracy
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == fake_labels).sum().item() / BATCH_SIZE * 100

    print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}, Accuracy = {accuracy:.1f}%")

print("\n✓ Training loop works!")

## 6. Experiment Ideas

Once everything works, try these experiments:

### A. Change the architecture
```python
# Try 3 conv layers instead of 2
self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
```

### B. Add Dropout (reduces overfitting)
```python
self.dropout = nn.Dropout(0.5)
# Use in forward: x = self.dropout(F.relu(self.fc1(x)))
```

### C. Add Batch Normalization (faster training)
```python
self.bn1 = nn.BatchNorm2d(32)
# Use in forward: x = self.pool(F.relu(self.bn1(self.conv1(x))))
```

### D. Try different learning rates
```python
LEARNING_RATE = 0.0001  # Slower but more stable
LEARNING_RATE = 0.01    # Faster but might be unstable
```

### E. Add Data Augmentation
```python
transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, translate=(0.1, 0.1)),
    # ... rest of transforms
])
```

## 7. Next Steps

Once your experiments work here:

1. ✅ Copy working code to `src/dataset.py` and `src/model.py`
2. ✅ Implement `src/train.py` with the full training loop
3. ✅ Move to `03_final_training.ipynb` for the real training
4. ✅ Celebrate your first AI model! 🎉